In [2]:
import numpy as np

In [3]:
np.random.seed(42)

In [4]:
x = (np.random.uniform(0,15,size=[1000,600])).astype(np.float64)
y = (np.random.uniform(0,15,size=[1000,600])).astype(np.float64)

# Distancias entre vectores

* [Distancia coseno](http://reference.wolfram.com/language/ref/CosineDistance.html):
$$
CosineDistance[\vec{u},\vec{v}] = 1 - \frac{\vec{u}\cdot\vec{v}}{\lVert \vec{u} 
\lVert \lVert \vec{v} \lVert}
$$

---

* [Distancia euclidiana](http://reference.wolfram.com/language/ref/EuclideanDistance.html):
$$
EuclidianDistance[\vec{u},\vec{v}] = \sqrt{ \sum_{k=1}^n (u_i-v_i)^2}
$$

In [5]:
def get_distance(vector, matrix,distance="cos"):
    
    # Distancia coseno
    if distance == "cos":
        '''return list(enumerate(1 - (np.matmul(vector, matrix.T) / 
                    (np.linalg.norm(vector) * 
                     np.sqrt(np.einsum('ij,ij->i',
                                       matrix, matrix))))))'''
        return [(i,1 - ((np.dot(vector,matrix[i]))/(np.linalg.norm(vector)*np.linalg.norm(matrix[i]))))
                for i in range(matrix.shape[0])]
    
    # Distancia euclidiana
    return [(i,np.linalg.norm(vector-matrix[i]))
            for i in range(matrix.shape[0])]

In [6]:
import time
c = None
tic = time.time()
c = ((1 - (np.matmul(x[0], x.T) / 
                    (np.linalg.norm(x[0]) * 
                     np.sqrt(np.einsum('ij,ij->i',
                                       x, x))))))
toc = time.time()
#print(type(c))
print(c[:20])
print((toc-tic)*1000)

tic = 0
toc = 0
del c
tic = time.time()
c = [(i,1 - ((np.dot(x[0],x[i]))/(np.linalg.norm(x[0])*np.linalg.norm(x[i]))))
                for i in range(x.shape[0])]
toc = time.time()
#print(type(c))
print(c[:20])
print(1000*(toc-tic))

[  1.11022302e-16   2.59753320e-01   2.52808239e-01   2.61176536e-01
   2.66679800e-01   2.67385099e-01   2.71955619e-01   2.42575630e-01
   2.52369195e-01   2.60923664e-01   2.49629194e-01   2.75721797e-01
   2.59722137e-01   2.62743879e-01   2.58569523e-01   2.42355492e-01
   2.72420748e-01   2.63215339e-01   2.64001494e-01   2.54203258e-01]
7.6694488525390625
[(0, 1.1102230246251565e-16), (1, 0.2597533198116333), (2, 0.2528082392094142), (3, 0.26117653630366966), (4, 0.2666797998707906), (5, 0.2673850985781554), (6, 0.27195561864309081), (7, 0.24257562992888537), (8, 0.25236919456259954), (9, 0.26092366404560885), (10, 0.24962919362038705), (11, 0.27572179734451352), (12, 0.25972213690435841), (13, 0.26274387861423754), (14, 0.25856952342955197), (15, 0.2423554923894512), (16, 0.27242074797865101), (17, 0.2632153392305755), (18, 0.26400149378022442), (19, 0.25420325844310121)]
23.555755615234375


In [7]:
dist = get_distance(x[0],x)

In [8]:
dist[:10]

[(0, 1.1102230246251565e-16),
 (1, 0.2597533198116333),
 (2, 0.2528082392094142),
 (3, 0.26117653630366966),
 (4, 0.2666797998707906),
 (5, 0.2673850985781554),
 (6, 0.27195561864309081),
 (7, 0.24257562992888537),
 (8, 0.25236919456259954),
 (9, 0.26092366404560885)]

In [9]:
dist = get_distance(x[0],x)

In [10]:
dist[:10]

[(0, 1.1102230246251565e-16),
 (1, 0.2597533198116333),
 (2, 0.2528082392094142),
 (3, 0.26117653630366966),
 (4, 0.2666797998707906),
 (5, 0.2673850985781554),
 (6, 0.27195561864309081),
 (7, 0.24257562992888537),
 (8, 0.25236919456259954),
 (9, 0.26092366404560885)]

In [11]:
dist.sort(key=lambda tup: tup[1])

In [12]:
dist[:10]

[(0, 1.1102230246251565e-16),
 (655, 0.22131986314650653),
 (743, 0.22164241690762032),
 (439, 0.22290881346365898),
 (541, 0.22321213563635356),
 (154, 0.22399918572998512),
 (685, 0.22403108889417234),
 (279, 0.22556434127799874),
 (341, 0.22589991553534927),
 (209, 0.22660554741884442)]